In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\sridhar\\Desktop\\DATA ANALYTICS PROJECTS\\DATA SCIENCE\\End-To-End-ML-Project'

In [14]:
# Purpose: Definition of the ModelTrainerConfig data class.

from dataclasses import dataclass  # Importing the dataclass decorator.
from pathlib import Path  # Object-oriented interface to filesystem paths.

@dataclass(frozen=True)
class ModelTrainerConfig:
    """
    Data class for configuration related to model training.

    Attributes:
        root_dir (Path): Root directory for model training artifacts.
        train_data_path (Path): Path to the training data CSV file.
        test_data_path (Path): Path to the test data CSV file.
        model_name (str): File name for saving the trained machine learning model.
        alpha (float): Regularization strength for models that support regularization.
        l1_ratio (float): The mixing parameter for elastic net regularization.
        target_column (str): Name of the target column in the dataset.
    """
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str


In [4]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

In [15]:
# Purpose: Definition of the ConfigurationManager class for managing project configurations.

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        """
        Initializes the ConfigurationManager with default or provided file paths.

        Args:
            config_filepath (Path, optional): Path to the configuration file. Defaults to CONFIG_FILE_PATH.
            params_filepath (Path, optional): Path to the parameters file. Defaults to PARAMS_FILE_PATH.
            schema_filepath (Path, optional): Path to the schema file. Defaults to SCHEMA_FILE_PATH.
        """
        # Reading configuration, parameters, and schema files using read_yaml function.
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Creating the root directory for project artifacts.
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        """
        Retrieves the configuration for model training.

        Returns:
            ModelTrainerConfig: Data class containing model training configuration.
        """
        # Extracting model trainer configuration, ElasticNet parameters, and target column from the overall project configuration.
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        # Creating the root directory for model training artifacts.
        create_directories([config.root_dir])

        # Creating a ModelTrainerConfig object with the extracted configuration.
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_config


In [6]:
import pandas as pd
import os
from mlproject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [17]:
# Purpose: Definition of the ModelTrainer class for training a machine learning model.

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        """
        Initializes the ModelTrainer object with the provided configuration.

        Args:
            config (ModelTrainerConfig): ModelTrainerConfig object containing model training settings.
        """
        self.config = config

    def train(self):
        """
        Trains a machine learning model using ElasticNet regression.
        """
        # Reading training and test data from CSV files.
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Extracting features and target variables from the data.
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        # Creating an ElasticNet regression model with specified alpha and l1_ratio.
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)

        # Training the model on the training data.
        lr.fit(train_x, train_y)

        # Saving the trained model using joblib in the specified root directory.
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [18]:
try:
    # Creating ConfigurationManager to manage project configurations.
    config = ConfigurationManager()

    # Retrieving model trainer configuration.
    model_trainer_config = config.get_model_trainer_config()

    # Creating ModelTrainer object with the obtained configuration.
    model_trainer = ModelTrainer(config=model_trainer_config)

    # Training the machine learning model.
    model_trainer.train()

except Exception as e:
    # Handling and re-raising any exceptions that occur during the process.
    raise e


[2023-12-02 22:27:23,921: INFO: common: YAML file: config\config.yaml loaded successfully]
[2023-12-02 22:27:23,933: INFO: common: YAML file: params.yaml loaded successfully]
[2023-12-02 22:27:23,938: INFO: common: YAML file: schema.yaml loaded successfully]
[2023-12-02 22:27:23,945: INFO: common: Created directory at: artifacts]
[2023-12-02 22:27:23,945: INFO: common: Created directory at: artifacts/model_trainer]
